In [1]:
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt

In [2]:
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
from os import listdir

2025-10-22 20:52:52.834803: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-22 20:52:52.888519: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-22 20:52:52.888581: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-22 20:52:52.892647: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-22 20:52:52.907586: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-22 20:52:52.908765: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
damagepath = "/code/Project2/COE379L-Project2/damage"
nodamagepath = "/code/Project2/COE379L-Project2/no_damage"

X = []
y = []

# Load damaged images and convert to black and white
for img_name in os.listdir(damagepath):
    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(damagepath, img_name)
        img = load_img(img_path)
        img_gray = img.convert('L')
        img_array = img_to_array(img_gray)
        X.append(img_array)
        y.append(1)

# Load non-damaged images
for img_name in os.listdir(nodamagepath):
    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(nodamagepath, img_name)
        img = load_img(img_path)
        img_gray = img.convert('L')
        img_array = img_to_array(img_gray)
        X.append(img_array)
        y.append(0) 

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

print("Loaded images:", X.shape)
print("Labels:", y.shape)

Loaded images: (21322, 128, 128, 1)
Labels: (21322,)


In [4]:
print("Total images:", X.shape)
print("Total labels:", y.shape)

Total images: (21322, 128, 128, 1)
Total labels: (21322,)


In [5]:
print("Image shape (Height x Width x Channels):", X[0].shape)
print("All image shapes unique:", np.unique([img.shape for img in X], axis=0))

Image shape (Height x Width x Channels): (128, 128, 1)
All image shapes unique: [[128 128   1]]


In [6]:
print("Min pixel value:", X.min())
print("Max pixel value:", X.max())

Min pixel value: 0.0
Max pixel value: 255.0


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 43, stratify = y)

In [9]:
# Now I will scale the image to a 1D array and Normalize the Data
image_vector_size = 128 * 128
X_train = X_train.reshape(X_train.shape[0], image_vector_size)
X_test = X_test.reshape(X_test.shape[0], image_vector_size)
X_train_normalized = X_train / 255.0
X_test_normalized = X_test / 255.0


In [9]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
# Here we map all the inputs to every perceptron (1024) creating a dense layer while also having 128*128 perceptrons in the first layer
model.add(Dense(256, input_dim=image_vector_size, activation='relu'))

# Here we create another dense layer except with less perceptrons
model.add(Dense(128, activation='relu'))

# Here we create another dense layer except with less perceptrons
model.add(Dense(64, activation='relu'))

# This is the output layer. The reason it only has 1 perceptron is because we are considering 0 to be 
# a non-destructed building and 1 to be a destructed building
model.add(Dense(1, activation='sigmoid'))

In [10]:
from keras.callbacks import EarlyStopping
# Creating an early stopping condition for the model
# This helps optimize the model
# In this case we look at val_loss (monitor)
# min_delta tells you the improvement required to not stop (.5 = if the epoch results in val_loss improving by < .5, then flag)
# patience tells us how long to continue doing epochs in a row if the flag is raised (flag not raising x times in a row). it then quits
# start_from_epoch tells us when to consider the early stopping
# restore_best_weights tells us to go to the best epoch when done
early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=0, start_from_epoch=5, restore_best_weights=True, verbose=1)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_normalized, y_train, validation_split=0.2, epochs=30, batch_size=4, verbose=2, callbacks=[early_stopping])

Epoch 1/30


2025-10-22 20:53:09.609903: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 894238720 exceeds 10% of free system memory.


3412/3412 - 67s - loss: 0.6614 - accuracy: 0.6566 - val_loss: 0.6460 - val_accuracy: 0.6527 - 67s/epoch - 20ms/step
Epoch 2/30
3412/3412 - 58s - loss: 0.6366 - accuracy: 0.6676 - val_loss: 0.6460 - val_accuracy: 0.6527 - 58s/epoch - 17ms/step
Epoch 3/30
3412/3412 - 54s - loss: 0.6367 - accuracy: 0.6676 - val_loss: 0.6462 - val_accuracy: 0.6527 - 54s/epoch - 16ms/step
Epoch 4/30
3412/3412 - 55s - loss: 0.6365 - accuracy: 0.6676 - val_loss: 0.6458 - val_accuracy: 0.6527 - 55s/epoch - 16ms/step
Epoch 5/30
3412/3412 - 60s - loss: 0.6365 - accuracy: 0.6676 - val_loss: 0.6461 - val_accuracy: 0.6527 - 60s/epoch - 18ms/step
Epoch 6/30
3412/3412 - 53s - loss: 0.6364 - accuracy: 0.6676 - val_loss: 0.6472 - val_accuracy: 0.6527 - 53s/epoch - 16ms/step
Epoch 7/30
Restoring model weights from the end of the best epoch: 6.
3412/3412 - 52s - loss: 0.6363 - accuracy: 0.6676 - val_loss: 0.6478 - val_accuracy: 0.6527 - 52s/epoch - 15ms/step
Epoch 7: early stopping


In [11]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Test Loss: 0.6383185982704163
Test Accuracy: 0.6647127866744995
